In [323]:
import pandas as pd

# Analiza receptov v računalniški igri Minecraft

### Uvod in razlage pojmov

Ta projekt iz glavne wiki spletne strani za računalniško igro Minecraft postrga vse recepte za izdelavo predmetov. Te nato shrani v CSV tabelo in jih analizira, a o tem kasneje.

Najprej se dogovorimo za nekaj izrazov:
- Material - predmet, ki je uporabljen v izdelavi drugega predmeta
- Predmet - predmet, ki ga poskušamo sestaviti iz materialov
- Recept - razporeditev do devetih matrialov v kvadratu velikosti 3x3, iz česar nato lahko dobimo nov predmet. Nam ni pomembna sama razporeditev, le koliko je potrebnega vsakega materiala

Prvi korak je naložiti HTML spletne strani na računalnik in ga predelati v datoteko CSV. To naredimo tako, da zaženemo program `main.py`, a sem na repozitorij objavil vse vmesne datoteke, saj je del programa, ki naloži HTML nekoliko nezanesljiv (zelo odvisen od internetne povezave, verzije iskalnika Chrome ipd.). Če vam pri zagonu `main.py` selenium vrača napako, nastavite spremenljivko na začetku programa na `download_from_web = False`.

In [324]:
csv_data = pd.read_csv("data/data.csv", header=0).sort_values("Name").sort_values("Section")
csv_data

,Section,Name,Ingredients,Amounts,Yield
344,Brewing,Fermented_Spider_Eye,Brown_Mushroom;Spider_Eye;Sugar,1;1;1,1
343,Brewing,Cauldron,Iron_Ingot,7,1
342,Brewing,Brewing_Stand,Blaze_Rod;Cobblestone,1;2,1
345,Brewing,Glass_Bottle,Glass,3,3
341,Brewing,Blaze_Powder,Blaze_Rod,1,2
...,...,...,...,...,...
308,Utilities,Clock,Gold_Ingot;Redstone_Dust,4;1,1
309,Utilities,Compass,Iron_Ingot;Redstone_Dust,4;1,1
310,Utilities,Lead,Slimeball;String,1;4,2
311,Utilities,Recovery_Compass,Compass;Echo_Shard,1;8,1


Poglejmo si tudi, kateri recepti proizvedejo največ predmetov oz. imajo največji donos:

In [325]:
csv_data.sort_values("Yield", ascending=False)

,Section,Name,Ingredients,Amounts,Yield
279,Transportation,Rail,Iron_Ingot;Stick,6;1,16
229,Decoration_blocks,Stained_Glass_Pane,White_Stained_Glass,6,16
188,Decoration_blocks,Glass_Pane,Glass,6,16
195,Decoration_blocks,Iron_Bars,Iron_Ingot,6,16
360,Materials,Gold_Nugget,Gold_Ingot,1,9
...,...,...,...,...,...
271,Transportation,Oak_Boat,Oak_Planks,5,1
292,Tools,Brush,Copper_Ingot;Feather;Stick,1;1;1,1
301,Tools,Stone_Shovel,Cobblestone;Stick,1;2,1
303,Tools,Warped_Fungus_on_a_Stick,Fishing_Rod;Warped_Fungus,1;1,1


Vidimo, da imajo železne rešetke, tračnice in vitraži ter prozorne steklene plošče vse najvišji donos, kar je 16 iz enega recepta.

### Primarni in sekundarni materiali

Sedaj bi radi definirali razliko med materiali: tisti, ki jih lahko naredimo iz česa drugega in tisti, ki jih ne moremo (na primer drva, kamen ipd.). Za to bomo definirali razred, ki bo hranil podatke o predmetih, da jih lahko nato primerjamo:

In [326]:
class Item():
    def __init__(self, section: str, name: str) -> None:
        self.name = name
        self.section = section
        self.recipes = []
        self.material_for = set()
        self.is_primary = False
        self.is_final = False
        self.max_depth = 0
        self.primary_ingredients = []
        # For each primary material, there is a tuple in the form of (Ingredient, Amount)
        self.items_crafted = []
        # A list of all the items, that can be crafted from this (directly or indirectly)
        self.recipe_chains = []
        # Contains lists of items, ranging from this item to its primary materials
        self.max_depth = 0
    
    def new_recipe(self, ings: str, ams: str, y: str):
        recipe = {}
        ings = ings.split(";")
        ams = ams.split(";")
        for i in range(len(ams)):
            recipe[ings[i]] = ams[i]
        recipe["yield"] = y
        self.recipes.append(recipe)
    
    def new_material(self, item: str):
        self.material_for.add(item)
    
    def __str__(self) -> str:
        out = self.name + f" is {"final" if self.is_final else ("primary" if self.is_primary else "intermediate")}"
        out += "\nItems crafted from this are: "
        for item in self.items_crafted:
            out += f"{item}, "
        out = out[:-2] + "\nThe necessary primary materials for this item are: (material, amount per produced item)\n"
        for material in self.primary_ingredients:
            out += f"({material[0]}, {str(round(material[1], 3))})\n"
        out += f"The longest chain of crafting operations for this item is {self.max_depth} operations long.\n"
        out += "#" * 50
        return out


Sedaj shranimo vsak predmet v CSV tabeli v svoj predmet razreda Item in te shranimo v slovar, da lahko lažje dostopamo do njih:

In [327]:
items = {}

for i, row in csv_data.iterrows():
    #print(row)
    items[row["Name"]] = Item(row["Section"], row["Name"])
    items[row["Name"]].new_recipe(row["Ingredients"], row["Amounts"], row["Yield"])

Vsak predmet razreda Item hrani podatke o specifičnem predmetu iz CSV datoteke in potem še dodaten podatek: za katere predmete je uporabljen.

Pri iskanju primarnih materialov je še ena ovira; nekatere primarne materiale je mogoče "stisniti" v kocko tega materiala, to kocko pa je potem mogoče razdreti nazaj na prvotno količino materiala (Na primer železo: 9 palic železa lahko spremenimo v kocko železa, to pa lahko spremenimo nazaj v 9 železnih palic).

Takšnih materialov je več, tako da jih poiščimo. Za vsak predmet bomo iskali, če je ta predmet material za katerega od svojih materialov:

In [328]:
for item in items.values():
    for recipe in item.recipes:
        for ingredient in recipe.keys():
            if ingredient not in items.keys():
                continue
            for recipe2 in items[ingredient].recipes:
                if item.name in recipe2.keys():
                    print(item.name)

Hay_Bale
Bone_Block
Block_of_Gold
Block_of_Iron
Slime_Block
Honey_Block
Honey_Bottle
Wheat
Bone_Meal
Iron_Ingot
Gold_Ingot
Slimeball
Block_of_Redstone
Redstone_Dust


Sedaj izmed teh izberemo tiste, ki jih nastavimo kot primarne:

In [329]:
primaries = ["Dried_Kelp", "Raw_Iron", "Raw_Gold", "Wheat", "Raw_Copper",
             "Coal", "Diamond", "Iron_Ingot", "Gold_Ingot", "Emerald", "Bone_Meal",
             "Lapis_Lazuli", "Netherite_Ingot", "Slimeball", "Honey_Block", "Redstone_Dust"]

Sedaj, ko imamo vse predmete shranjene, lahko za vsak predmet preverimo, v katerem receptu je material za drug predmet:

In [330]:
materials = {}

for item in items.keys():
    for recipe in items[item].recipes:
        for material in recipe.keys():
            if material == "yield":
                continue
            if material not in items.keys():
                # Če material v receptu še ni v slovarju predmetov, to pomeni,
                # da nima recepta, ki bi naredil ta predmet, torej je primarni material
                materials[material] = Item("Primary", material)
                materials[material].is_primary = True
                if material not in primaries:
                    primaries.append(material)
                materials[material].new_material(item)
            elif material in primaries:
                items[material].is_primary = True
            # Vsakemu materialu v vsakem receptu dodamo v seznam predmetov,
            # ki se jih da narediti iz tega materiala, predmet, ki je posledica tega recepta
            else:
                items[material].new_material(item)
items = materials | items

Preverimo, da program pravilno zazna vse primarne materiale (če predmet nima recepta in je v receptu, potem je primarni material):

In [331]:
print("Predmeti, ki nimajo recepta in niso primarni:")
for item in items.keys():
    if len(items[item].recipes) == 0 and not items[item].is_primary:
        print(item, items[item].material_for)
print("Konec!")

Predmeti, ki nimajo recepta in niso primarni:
Konec!


Ker je presek med množico predmetov, ki niso primarni materiali, in množico predmetov brez recepta, prazna, naš program deluje pravilno.

Sedaj lahko zelo preprosto tudi definiramo končne predmete, to so tisti predmeti, ki niso material za noben drug predmet:

In [332]:
for item in items.keys():
    if len(items[item].material_for) == 0:
        items[item].is_final = True

Tega ne bomo preverjali, saj precej očitno deluje.

## Veriga receptov

Sedaj, ko imamo te podatke na voljo, lahko tvorimo verige receptov. Zanima nas, Katere vse materiale moramo narediti, da izdelamo vsak predmet. Poleg tega bi bilo priročno vedeti, koliko katerih primarnih materialov potrebujemo za poljuben predmet, in za katere predmete moramo uporabiti posamezen primarni element.

In [333]:
def recursive_recipe_search(name: str, multiplier: float=1) -> list:
    if items[name].is_primary:
        return []
    out = []
    for i, recipe in enumerate(items[name].recipes):
        out.append([])
        if len(recipe.keys()) == 0:
            continue
        for material in recipe.keys():
            if material == None:
                return []
            if material == "yield":
                continue
            mult = multiplier * int(recipe[material]) / int(recipe["yield"])
            if mult == None:
                return []
            out[i].append((material, mult))
            out[i] += recursive_recipe_search(material, mult)
    return out

for item in items.values():
    if not item.is_primary:
        item.recipe_chains = recursive_recipe_search(item.name)
        print(item.name, ":", item.recipe_chains)

Fermented_Spider_Eye : [[('Brown_Mushroom', 1.0), ('Spider_Eye', 1.0), ('Sugar', 1.0), [('Honey_Bottle', 0.3333333333333333), [('Glass_Bottle', 1.3333333333333333), [('Glass', 1.3333333333333333)], ('Honey_Block', 0.3333333333333333)]]]]
Cauldron : [[('Iron_Ingot', 7.0)]]
Brewing_Stand : [[('Blaze_Rod', 1.0), ('Cobblestone', 2.0)]]
Glass_Bottle : [[('Glass', 1.0)]]
Blaze_Powder : [[('Blaze_Rod', 0.5)]]
Magma_Cream : [[('Blaze_Powder', 1.0), [('Blaze_Rod', 0.5)], ('Slimeball', 1.0)]]
Glistering_Melon_Slice : [[('Gold_Nugget', 8.0), [('Gold_Ingot', 0.8888888888888888)], ('Melon_Slice', 1.0)]]
Polished_Diorite_Slab : [[('Polished_Diorite', 0.5), [('Diorite', 0.5), [('Cobblestone', 0.5), ('Nether_Quartz', 0.5)]]]]
Deepslate_Tiles : [[('Deepslate_Bricks', 1.0), [('Polished_Deepslate', 1.0), [('Cobbled_Deepslate', 1.0)]]]]
Stone_Brick_Stairs : [[('Stone_Bricks', 1.5), [('Stone', 1.5)]]]
Diorite : [[('Cobblestone', 1.0), ('Nether_Quartz', 1.0)]]
Diorite_Slab : [[('Diorite', 0.5), [('Cobblesto

Sedaj imamo za vsak predmet verigo materialov v obliki gnezdenih seznamov. Iz tega lahko preberemo, katere primarne materiale potrebujemo (in koliko vsakega) za vsak predmet in koliko korakov potrebujemo, da pridemo od primarnih materialov do želenega predmeta:

In [334]:
def find_primaries_and_depth(recipe_chain: list, depth: int=0):
    out = []
    for link in recipe_chain:
        if type(link) == type([]):
            temp = find_primaries_and_depth(link)
            out += temp[0]
            depth = temp[1]
        else:
            if items[link[0]].is_primary:
                out.append(link)
    
    return out, depth + 1

for item in items.values():
    out = []
    for chain in item.recipe_chains:
        temp = find_primaries_and_depth(chain)
        out += temp[0]
        item.max_depth = max(item.max_depth, temp[1])
    for i, tup in enumerate(out):
        for j in range(i):
            if out[j][0] == tup[0]:
                if out[j][1] > tup[1]:
                    out.pop(i)
                else:
                    out.pop(j)
    for mat, amt in out:
        amt = float.as_integer_ratio(amt)
    item.primary_ingredients = out
    print(item.name, item.max_depth, item.primary_ingredients)

Brown_Mushroom 0 []
Spider_Eye 0 []
Blaze_Rod 0 []
Cobblestone 0 []
Glass 0 []
Melon_Slice 0 []
Nether_Quartz 0 []
Pointed_Dripstone 0 []
Dried_Kelp 0 []
Nether_Brick 0 []
Gravel 0 []
Sand 0 []
Nautilus_Shell 0 []
Popped_Chorus_Fruit 0 []
Stone 0 []
Prismarine_Shard 0 []
Nether_Wart 0 []
End_Stone 0 []
Terracotta 0 []
Smooth_Red_Sandstone 0 []
Smooth_Quartz_Block 0 []
Carved_Pumpkin 0 []
Red_Sand 0 []
Prismarine_Crystals 0 []
Bamboo 0 []
String 0 []
Smooth_Sandstone 0 []
Stripped_Oak_Log 0 []
Moss_Block 0 []
Snowball 0 []
Smooth_Stone 0 []
Ice 0 []
Raw_Iron 0 []
Raw_Gold 0 []
Basalt 0 []
Cut_Copper_Slab 0 []
Honeycomb 0 []
Cut_Copper 0 []
Oak_Planks 0 []
Bamboo_Mosaic 0 []
Cut_Copper_Stairs 0 []
Raw_Copper 0 []
Blackstone 0 []
Polished_Blackstone 0 []
Amethyst_Shard 0 []
Cobbled_Deepslate 0 []
Coal 0 []
Copper_Ingot 0 []
Diamond 0 []
Emerald 0 []
Lapis_Lazuli 0 []
Oak_Log 0 []
Prismarine 0 []
Red_Sandstone_Slab 0 []
Sandstone_Slab 0 []
Tuff_Slab 0 []
Tuff_Brick_Slab 0 []
Dirt 0 []
Bric

Nazadnje poglejmo še, katere predmete lahko (posredno ali neposredno) naredimo iz vsakega predmeta:

In [335]:
def recursive_material_search(item: str):
    out = set()
    for material in items[item].material_for:
        out.add(material)
        if not items[material].is_final:
            out.union(recursive_material_search(material))
    return out

for item in items.keys():
    items[item].items_crafted = recursive_material_search(item)
    print(item, items[item].items_crafted)

Brown_Mushroom {'Mushroom_Stew'}
Spider_Eye {'Fermented_Spider_Eye'}
Blaze_Rod {'End_Rod'}
Cobblestone {'Stone_Pickaxe'}
Glass {'Daylight_Detector'}
Melon_Slice {'Melon_Seeds'}
Nether_Quartz {'Observer'}
Pointed_Dripstone {'Dripstone_Block'}
Dried_Kelp {'Dried_Kelp_Block'}
Nether_Brick {'Nether_Brick_Fence'}
Gravel {'Coarse_Dirt'}
Sand {'Sandstone'}
Nautilus_Shell {'Conduit'}
Popped_Chorus_Fruit {'End_Rod'}
Stone {'Redstone_Repeater'}
Prismarine_Shard {'Prismarine_Bricks'}
Nether_Wart {'Red_Nether_Bricks'}
End_Stone {'End_Stone_Bricks'}
Terracotta {'Stained_Terracotta'}
Smooth_Red_Sandstone {'Smooth_Red_Sandstone_Slab'}
Smooth_Quartz_Block {'Smooth_Quartz_Slab'}
Carved_Pumpkin {"Jack_o'Lantern"}
Red_Sand {'Red_Sandstone'}
Prismarine_Crystals {'Sea_Lantern'}
Bamboo {'Stick'}
String {'Lead'}
Smooth_Sandstone {'Smooth_Sandstone_Stairs'}
Stripped_Oak_Log {'Hanging_Sign'}
Moss_Block {'Moss_Carpet'}
Snowball {'Snow_Block'}
Smooth_Stone {'Blast_Furnace'}
Ice {'Packed_Ice'}
Raw_Iron {'Block_of

Sedaj dodamo razredu Item funkcijo `__str__`, da lahko uporabimo `print(predmet)`, da vidimo vse nabrane podatke o predmetu:

In [336]:
for item in items.values():
    print(item)

Brown_Mushroom is primary
Items crafted from this are: Mushroom_Stew
The necessary primary materials for this item are: (material, amount per produced item)
The longest chain of crafting operations for this item is 0 operations long.
##################################################
Spider_Eye is primary
Items crafted from this are: Fermented_Spider_Eye
The necessary primary materials for this item are: (material, amount per produced item)
The longest chain of crafting operations for this item is 0 operations long.
##################################################
Blaze_Rod is primary
Items crafted from this are: End_Rod
The necessary primary materials for this item are: (material, amount per produced item)
The longest chain of crafting operations for this item is 0 operations long.
##################################################
Cobblestone is primary
Items crafted from this are: Stone_Pickaxe
The necessary primary materials for this item are: (material, amount per produced item)

Vse te podatke shranimo še v novo CSV datoteko:

In [338]:

out = "Item,Position,Primaries,Primary_Amounts,Crafts,Chain\n"
for item in items.values():
    out += item.name + ","
    out += "Primary" if item.is_primary else ("Final" if item.is_final else "Intermediate")
    out += ","
    if item.is_primary:
        out += "_,_,"
    else:
        for primary in item.primary_ingredients:
            out += primary[0] + ";"
        out = out[:-1] + ","
        for primary in item.primary_ingredients:
            out += str(round(primary[1], 3)) + ";"
        out = out[:-1] + ","
    if item.is_final:
        out += "_,\n"
    else:
        for craft in item.items_crafted:
            out += craft
            out += ";"
        out = out[:-1] + ","
    out += str(item.max_depth)
    out += "\n"
out.strip("\n")
with open("data/extracted.csv", "w", encoding="utf8") as file:
    file.write(out)
    

Nazadnje pa si poglejmo še nekaj zanimivih grafov: